In [126]:
# https://en.wikipedia.org/wiki/Taylor%27s_theorem#Example_in_two_dimensions
from sympy import *
x_i, x_j, betaY2 = symbols('x_i, x_j, betaY2')

fYij = x_i*x_j*(x_i+x_j)/(betaY2*x_i + x_j)
display(fYij)

def dosub(x):
    return x.subs(x_i, 1e-10).subs(x_j,3e-10).subs(betaY2, 1.1)
def sub0(x):
    return x.subs([(x_i, 0), (x_j,0)])
def lim0(x):
    return limit(x.subs([(x_i, x_j), ]), x_j, 0)

def chk0(x):
    L = lim0(x)
    S = sub0(x)
    if L != S:
        print('Derivatives do not agree:', L, S)
    assert(L==S)
    return L

val = dosub(fYij)
display(val)

deg1terms = [chk0(diff(fYij, x_i, 1))*x_i/factorial(1), chk0(diff(fYij, x_j, 1))*x_j/factorial(1)]
deg2terms = [chk0(diff(fYij, x_j, 2))*x_j**2/factorial(2), chk0(diff(diff(fYij, x_j, 1), x_i, 1))*x_j*x_i, chk0(diff(fYij, x_i, 2))*x_i**2/factorial(2)]
deg3terms = [
    chk0(diff(fYij, x_j, 3))*x_j**3/factorial(3)
  + chk0(diff(diff(fYij, x_j, 1), x_i, 2))*x_j*x_i**2/factorial(2)
  + chk0(diff(diff(fYij, x_j, 2), x_i, 1))*x_j**2*x_i/factorial(2) 
  + chk0(diff(fYij, x_i, 3))*x_i**3/factorial(3)
]
on1st = sum(deg1terms)
display(deg1terms)
on2nd = sum(deg2terms)
display(deg2terms)
on3rd = sum(deg3terms)
display(deg3terms)

x_i*x_j*(x_i + x_j)/(betaY2*x_i + x_j)

2.92682926829268e-20

Derivatives do not agree: 2*(betaY2**2 - betaY2)/((betaY2 + 1)*(betaY2**2 + 2*betaY2 + 1)) 0


AssertionError: 

In [2]:
def Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    Mathematical formulation reference:
    https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
    :param function_expression: Sympy expression of the function
    :param variable_list: list. All variables to be approximated (to be "Taylorized")
    :param evaluation_point: list. Coordinates, where the function will be expressed
    :param degree: int. Total degree of the Taylor polynomial
    :return: Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial

for i in [1,2,3]:
    display(Taylor_polynomial_sympy(fYij, [x_i, x_j], [0,0], i))
# dosub(Taylor_polynomial_sympy(fYij, [x_i, x_j], [0,0], 3))

0

x_i**2*(1 - betaY2) + x_i*x_j

zoo*betaY2*x_i**3*(betaY2 - 1) + x_i**2*(1 - betaY2) + x_i*x_j

In [88]:
def fYgerg(x_i, x_j, betaY2):
    return (
        (x_i+x_j)/(betaY2*x_i + x_j) 
        + x_j/(betaY2*x_i + x_j)*(1-(x_i+x_j)/(betaY2*x_i + x_j)) 
        + x_i/(betaY2*x_i + x_j)*(1-betaY2*(x_i+x_j)/(betaY2*x_i + x_j))
        - x_i*x_j/(betaY2*x_i + x_j)**2*(1 + betaY2 - 2*betaY2*(x_i+x_j)/(betaY2*x_i+x_j))
    )

display(fYgerg(1e-12, 1e-12, 1.1), diff(diff(fYij, x_j, 1), x_i, 1).subs([(x_i, 1e-12), (x_j, 1e-12), (betaY2, 1.1)]))
display(limit(fYgerg(x_i, x_i, betaY2), x_i, 0.0))
display(limit(fYgerg(x_j, x_j, betaY2), x_j, 0.0))

0.9513011553827879

0.951301155382788

(betaY2**2 + 6*betaY2 + 1)/(betaY2**3 + 3*betaY2**2 + 3*betaY2 + 1)

(betaY2**2 + 6*betaY2 + 1)/(betaY2**3 + 3*betaY2**2 + 3*betaY2 + 1)